### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR(pair-wise loss) на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP(list-wise loss) на implicit данных

Мягкий дедлайн 28 Сентября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 5 Октября (Итоговая проверка)

In [1]:
import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
from tqdm.notebook import tqdm

from lightfm.datasets import fetch_movielens

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [2]:
ratings = pd.read_csv('ml-1m/ratings.dat', delimiter='::', header=None, 
        names=['user_id', 'movie_id', 'rating', 'timestamp'], 
        usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [3]:
movie_info = pd.read_csv('ml-1m/movies.dat', delimiter='::', header=None, 
        names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [4]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [5]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [6]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [7]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [8]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

В качестве loss здесь всеми любимый RMSE

In [9]:
model.fit(user_item_t_csr)

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [10]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [12]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '2315    Babe: Pig in the City (1998)',
 '584    Aladdin (1992)',
 '1526    Hercules (1997)',
 '3817    Went to Coney Island on a Mission From God... ...',
 '2252    Pleasantville (1998)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [13]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [14]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [15]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [16]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '2502    Matrix, The (1999)',
 '3402    Close Encounters of the Third Kind (1977)',
 '847    Godfather, The (1972)',
 '3458    Predator (1987)',
 '2460    Planet of the Apes (1968)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

In [17]:
import torch
from torch.nn import functional as F
from sklearn.neighbors import KDTree 
import random

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [18]:
def ratings_to_tensor(ratings, unique_users=None, unique_movies=None):
    if unique_users is None:
        unique_users = ratings.user_id.unique()
    if unique_movies is None:
        unique_movies = ratings.movie_id.unique()
    user2id = dict([(u, i) for i, u in enumerate(unique_users)])
    movie2id = dict([(m, i) for i, m in enumerate(unique_movies)])
    mask = torch.zeros(len(unique_users), len(unique_movies))
    mat = torch.zeros(len(unique_users), len(unique_movies))
    for u, m, r in zip(ratings.user_id, ratings.movie_id, ratings.rating):
        mat[user2id[u]][movie2id[m]] = float(r)
        mask[user2id[u]][movie2id[m]] = 1
    mask = mask.bool()
    return mat, mask, unique_users, unique_movies

ratings_tensor, rt_mask, u_users, u_movies = ratings_to_tensor(ratings)

In [19]:
ratings_tensor

tensor([[5., 3., 3.,  ..., 0., 0., 0.],
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.]])

In [20]:
class RecSys:
    def __init__(self, users, items):
        self.user2id = dict([(u, i) for i, u in enumerate(users)])
        self.item2id = dict([(m, i) for i, m in enumerate(items)])
        self.users = users
        self.items = items
        self.items_tree = None
        
    def get_items_repr(self):
        raise NotImplemented
    
    def compute_score(self):
        raise NotImplemented
        
    def _build_trees(self):
        self.items_tree = KDTree(self.get_items_repr())
    
    def recommend(self, user_id, _, k=20):
        u = self.user2id[user_id]
        r = self.compute_score()[u]
        idx = list(reversed(np.argsort(r)))[:k]
        result_idx = [self.items[i] for i in idx]
        return list(zip(result_idx, r[idx]))
        
    def similar_items(self, item_id, k=20):
        i = [self.get_items_repr()[self.item2id[item_id]]]
        d, idx = self.items_tree.query(i, k, return_distance=True)
        true_idx = [self.items[i] for i in idx[0]]
        return list(zip(true_idx, d[0]))

In [21]:
class SVD(RecSys):
    def __init__(self, users, items, hidden_size=64):
        super().__init__(users, items)
        self.U = torch.rand(len(users), hidden_size) / hidden_size**0.5
        self.V = torch.rand(hidden_size, len(items)) / hidden_size**0.5
        
    def __call__(self):
        return self.U.matmul(self.V)
    
    def get_items_repr(self):
        return self.V.transpose(0, 1).cpu().numpy()
    
    def compute_score(self):
        return self().cpu().numpy()

    def fit(self, x, x_mask, iterations=500, lr=100.0, weight_decay=1e-2, masked=False):
        # It can be done way more effectively with good dataloader, batches and torch.optim, 
        # but ready solutions are forbidden :(
        t = tqdm(range(iterations))
        ratings_count = x_mask.int().sum().item()
        u_count = torch.ones_like(self.U).sum() # Well, this is obviously not optimal
        v_count = torch.ones_like(self.V).sum()
        for i in t:
            eps = (self() - x)
            eps[x_mask.logical_not()] = 0 # Do not update over NaN items
            u_grad = eps.matmul(self.V.transpose(0, 1)) / ratings_count + weight_decay * self.U / u_count
            v_grad = self.U.transpose(0, 1).matmul(eps) / ratings_count + weight_decay * self.V / v_count
            self.U -= lr * u_grad
            self.V -= lr * v_grad
            mse = ((self() - x) ** 2)[x_mask].sum() / ratings_count
            t.set_postfix_str(f"MSE: {mse:.4f} | L2 Norm: {(self.U**2).mean():.4f}")
        self._build_trees()

svd = SVD(u_users, u_movies, 64)
svd.fit(ratings_tensor, rt_mask)

In [22]:
ratings_tensor

tensor([[5., 3., 3.,  ..., 0., 0., 0.],
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.]])

In [23]:
svd()

tensor([[4.5910, 3.6594, 4.3366,  ..., 0.8262, 1.2549, 1.3526],
        [4.1528, 3.2706, 4.0347,  ..., 0.7396, 1.1527, 1.2397],
        [4.5018, 3.4541, 4.3040,  ..., 0.7892, 1.2092, 1.3049],
        ...,
        [4.1187, 3.2485, 3.9589,  ..., 0.7326, 1.1438, 1.1962],
        [4.3108, 3.3682, 4.0602,  ..., 0.7299, 1.1456, 1.2159],
        [4.2949, 3.4033, 3.7430,  ..., 0.6696, 1.0649, 1.1771]])

In [24]:
get_recommendations(4, svd)

['2836    Sanjuro (1962)',
 '847    Godfather, The (1972)',
 '1132    Wrong Trousers, The (1993)',
 '1950    Seven Samurai (The Magnificent Seven) (Shichin...',
 '1162    Paths of Glory (1957)',
 '900    Casablanca (1942)',
 '1189    To Kill a Mockingbird (1962)',
 "523    Schindler's List (1993)",
 '49    Usual Suspects, The (1995)',
 '892    Rear Window (1954)',
 '735    Close Shave, A (1995)',
 '315    Shawshank Redemption, The (1994)',
 '910    Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)',
 '2953    General, The (1927)',
 '1230    Bridge on the River Kwai, The (1957)',
 "1176    One Flew Over the Cuckoo's Nest (1975)",
 '2961    Yojimbo (1961)',
 '1186    Lawrence of Arabia (1962)',
 '740    Dr. Strangelove or: How I Learned to Stop Worr...',
 '1242    Great Escape, The (1963)']

In [25]:
get_similars(1, svd)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '591    Beauty and the Beast (1991)',
 '360    Lion King, The (1994)',
 '1222    Glory (1989)',
 '2012    Little Mermaid, The (1989)',
 '33    Babe (1995)',
 '2338    Cocoon (1985)',
 '1282    Field of Dreams (1989)',
 '942    Mr. Smith Goes to Washington (1939)',
 '2009    Jungle Book, The (1967)',
 '436    Dave (1993)',
 '1016    Dumbo (1941)',
 '907    Wizard of Oz, The (1939)',
 '3294    American Graffiti (1973)',
 '148    Apollo 13 (1995)',
 "890    Breakfast at Tiffany's (1961)",
 '1421    Kolya (1996)']

### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [26]:
i_ratings_tensor, i_rt_mask, _, _ = ratings_to_tensor(implicit_ratings, u_users, u_movies)

In [27]:
class ALS(RecSys):
    def __init__(self, users, items, hidden_size=64):
        super().__init__(users, items)
        self.U = torch.rand(len(users), hidden_size) / hidden_size**0.5
        self.V = torch.rand(hidden_size, len(items)) / hidden_size**0.5
        self.v_bias = torch.zeros(len(items))
        self.u_bias = torch.zeros(len(users))
        self.hidden_size = hidden_size
        
    def __call__(self):
        return self.U.matmul(self.V)
    
    def get_items_repr(self):
        return self.V.transpose(0, 1).cpu().numpy()
    
    def compute_score(self):
        return self().cpu().numpy()

    def fit(self, x, x_mask, iterations=8, nonzero_coef=10, weight_decay=0.01):
        t = tqdm(range(iterations))
        ratings_count = x_mask.int().sum().item() 
        u_count = torch.ones_like(self.U).sum() # Well, this is obviously not optimal
        v_count = torch.ones_like(self.V).sum()
        x = x.clone()
        x[x_mask] = 1
        w = torch.ones_like(x)
        w[x_mask] += nonzero_coef * x[x_mask].abs()
        w /= w.max()
        x_target = w * x
        for _ in t:
            # Step 1: User representations
            for u in range(len(self.U)):
                Vb = torch.cat([self.V, torch.ones(1, self.V.size(1))], dim=0) # Add ones to include bias
                VCV = Vb.matmul(w[u][:, None] * Vb.transpose(0, 1))
                VCV = VCV + weight_decay * torch.eye(self.hidden_size + 1)
                VCV = torch.inverse(VCV)
                Ub = VCV.matmul(Vb).matmul((w[u] * (x[u] - self.v_bias)))
                self.U[u] = Ub[:-1]
                self.u_bias[u] = Ub[-1]
            # Step 2: Item representations
            for i in range(self.V.size(1)):
                Ub  = torch.cat([self.U, torch.ones(self.U.size(0), 1)], dim=1) # Add ones to include bias
                UCU = Ub.transpose(0, 1).matmul(w[:, i][:, None] * Ub)
                UCU = UCU + weight_decay * torch.eye(self.hidden_size + 1)
                UCU = torch.inverse(UCU)
                Vb = UCU.matmul(Ub.transpose(0, 1)).matmul((w[:, i] * (x[:, i] - self.u_bias)))
                self.V[:, i] = Vb[:-1]
                self.v_bias[i] = Vb[-1]
            # Log
            uv = self()
            mse = ((uv + self.u_bias[:, None].expand_as(uv)  + self.v_bias[None, :].expand_as(uv) - x) ** 2)[x_mask].sum() / ratings_count
            t.set_postfix_str(f"MSE: {mse:.4f} | L2 Norm: {(self.U**2).mean():.4f}")
        self._build_trees()

als = ALS(u_users, u_movies, 64)
als.fit(i_ratings_tensor, i_rt_mask)

In [28]:
i_ratings_tensor

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.]])

In [29]:
als()

tensor([[-0.0962,  0.1967,  0.0529,  ..., -0.2900, -0.2619, -0.3078],
        [ 0.3977, -0.2376,  0.0501,  ..., -0.2349, -0.2373, -0.2313],
        [-0.1559, -0.3170, -0.2624,  ..., -0.0858, -0.0743, -0.0843],
        ...,
        [-0.1803, -0.3090,  0.0109,  ..., -0.1518, -0.1719, -0.1570],
        [-0.0149,  0.4103,  0.9205,  ...,  0.0949,  0.0782,  0.0629],
        [ 0.5793,  0.1374,  0.3465,  ...,  0.0140, -0.0116,  0.0244]])

In [30]:
get_recommendations(4, als)

['1183    Good, The Bad and The Ugly, The (1966)',
 '1366    Jaws (1975)',
 '1220    Terminator, The (1984)',
 '2878    Goldfinger (1964)',
 '1196    Alien (1979)',
 '1885    Rocky (1976)',
 '1023    Die Hard (1988)',
 '585    Terminator 2: Judgment Day (1991)',
 '1182    Aliens (1986)',
 '453    Fugitive, The (1993)',
 '2882    Fistful of Dollars, A (1964)',
 '1267    Ben-Hur (1959)',
 '3349    Thelma & Louise (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '2502    Matrix, The (1999)',
 '3633    Mad Max (1979)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2875    Dirty Dozen, The (1967)',
 '2297    King Kong (1933)',
 '1180    Raiders of the Lost Ark (1981)']

In [31]:
get_similars(1, als)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 '1245    Groundhog Day (1993)',
 "2286    Bug's Life, A (1998)",
 '584    Aladdin (1992)',
 '33    Babe (1995)',
 '2252    Pleasantville (1998)',
 '2327    Shakespeare in Love (1998)',
 "1854    There's Something About Mary (1998)",
 '1596    Indian Summer (a.k.a. Alive & Kicking) (1996)',
 '990    Extreme Measures (1996)',
 '217    Cure, The (1995)',
 '634    Girl 6 (1996)',
 '2087    Best Man, The (Il Testimone dello sposo) (1997)',
 '1445    Best Men (1997)',
 '26    Now and Then (1995)',
 '355    I Like It Like That (1994)',
 '1938    Polish Wedding (1998)',
 '360    Lion King, The (1994)',
 '3092    Onegin (1999)']

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [32]:
def build_positives_and_negatives(x):
    user2positives = []
    user2negatives = []
    for u in tqdm(range(len(x))):
        user2positives.append((x[u] != 0).nonzero().view(-1).tolist())
        user2negatives.append((x[u] == 0).nonzero().view(-1).tolist())
    return user2positives, user2negatives

u2p, u2n = build_positives_and_negatives(i_rt_mask)

<ipython-input-32-9dacc2bb0e6c>:5: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  user2positives.append((x[u] != 0).nonzero().view(-1).tolist())


In [33]:
class BPR(RecSys):
    def __init__(self, users, items, hidden_size=64):
        super().__init__(users, items)
        self.U = torch.rand(len(users), hidden_size) / hidden_size**0.5
        self.V = torch.rand(hidden_size, len(items)) / hidden_size**0.5
        
    def __call__(self):
        return self.U.matmul(self.V)
    
    def get_items_repr(self):
        return self.V.transpose(0, 1).cpu().numpy()
    
    def compute_score(self):
        return self().cpu().numpy()

    def fit(self, user2positives, user2negatives, iterations=500, acc_grad=16, lr=0.1, weight_decay=1e-2, masked=False):
        # It can be done way more effectively with good dataloader, batches and torch.optim, 
        # but ready solutions are forbidden :(
        
        t = tqdm(range(iterations))
        for i in t:
            u_grad = acc_grad * weight_decay * self.U
            v_grad = acc_grad * weight_decay * self.V
            mean_delta = 0.
            x = self()
            for _ in range(acc_grad):
                # Build a batch
                positives = []
                negatives = []
                pos_mask = []
                neg_mask = []
                for u in range(len(user2positives)):
                    if len(user2positives[u]) != 0:
                        pos_mask.append(1)
                        positives.append(user2positives[u][random.randint(0, len(user2positives[u]) - 1)])
                    else:
                        pos_mask.append(0)
                        positives.append(0)
                    if len(user2negatives[u]) != 0:
                        neg_mask.append(1)
                        negatives.append(user2negatives[u][random.randint(0, len(user2negatives[u]) - 1)])
                    else:
                        neg_mask.append(0)
                        negatives.append(0)
                positives = torch.tensor(positives)
                negatives = torch.tensor(negatives)
                pos_mask = torch.tensor(pos_mask).float()
                neg_mask = torch.tensor(neg_mask).float()

                # Compute gradient
                delta = x.gather(1, positives.unsqueeze(1)) - x.gather(1, negatives.unsqueeze(1)) 
                mean_delta += delta.mean()
                delta = torch.exp(-delta).view(-1)
                delta = -delta / (1 + delta)
                v_positives = self.V[:, positives].transpose(0, 1)
                v_negatives = self.V[:, negatives].transpose(0, 1)
                u_grad += delta[:, None] * (pos_mask[:, None] * v_positives - neg_mask[:, None] * v_negatives)

                v_grad[:, positives] += ((pos_mask * delta)[:, None] * self.U).transpose(0, 1)
                v_grad[:, negatives] -= ((neg_mask * delta)[:, None] * self.U).transpose(0, 1)
            self.U -= lr * u_grad / acc_grad
            self.V -= lr * v_grad / acc_grad
            mean_delta /= acc_grad
            t.set_postfix_str(f"Avg delta: {mean_delta:4f} | Avg nabla V: {v_grad.abs().mean():.4f} | L2 Norm: {(self.U**2).mean():.4f}")
        self._build_trees()

bpr = BPR(u_users, u_movies, 64)
bpr.fit(u2p, u2n)

In [34]:
i_ratings_tensor

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.]])

In [35]:
bpr()

tensor([[-17.4593, -16.5712, -15.7135,  ..., -19.7635, -19.6285, -19.8437],
        [-17.2175, -19.1786, -17.7831,  ..., -20.1193, -20.0580, -20.2460],
        [-18.8126, -18.2768, -18.2640,  ..., -20.1706, -20.0336, -20.2300],
        ...,
        [-13.0497, -12.3906, -10.8588,  ..., -14.3123, -14.3344, -14.3234],
        [-14.4118, -14.1217, -12.2789,  ..., -16.8662, -16.8537, -16.8588],
        [-14.1867, -15.7780, -15.7144,  ..., -17.1913, -17.2094, -17.0888]])

In [36]:
get_recommendations(4, bpr)

['257    Star Wars: Episode IV - A New Hope (1977)',
 '1196    Alien (1979)',
 '2878    Goldfinger (1964)',
 '1366    Jaws (1975)',
 '2879    From Russia with Love (1963)',
 '1885    Rocky (1976)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1220    Terminator, The (1984)',
 '1023    Die Hard (1988)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '585    Terminator 2: Judgment Day (1991)',
 '2882    Fistful of Dollars, A (1964)',
 '1180    Raiders of the Lost Ark (1981)',
 '2297    King Kong (1933)',
 '1182    Aliens (1986)',
 '1267    Ben-Hur (1959)',
 '2875    Dirty Dozen, The (1967)',
 '847    Godfather, The (1972)',
 '2847    Total Recall (1990)',
 '2993    Longest Day, The (1962)']

In [37]:
get_similars(1, bpr)

['0    Toy Story (1995)',
 '2225    Antz (1998)',
 '584    Aladdin (1992)',
 '1005    That Darn Cat! (1965)',
 '33    Babe (1995)',
 '1850    Madeline (1998)',
 '2067    Nutty Professor, The (1963)',
 '463    Live Nude Girls (1995)',
 '2285    Rugrats Movie, The (1998)',
 '565    Little Big League (1994)',
 '1245    Groundhog Day (1993)',
 '1433    That Darn Cat! (1997)',
 '1664    Mouse Hunt (1997)',
 '1445    Best Men (1997)',
 '3327    Muppet Movie, The (1979)',
 '2759    Dudley Do-Right (1999)',
 '2315    Babe: Pig in the City (1998)',
 '2082    Gods Must Be Crazy II, The (1989)',
 '179    Mighty Morphin Power Rangers: The Movie (1995)',
 '1586    MatchMaker, The (1997)']

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных

In [59]:
class WARP(RecSys):
    def __init__(self, users, items, hidden_size=64):
        super().__init__(users, items)
        self.U = torch.rand(len(users), hidden_size) / hidden_size**0.5
        self.V = torch.rand(hidden_size, len(items)) / hidden_size**0.5
        
    def __call__(self):
        return self.U.matmul(self.V)
    
    def get_items_repr(self):
        return self.V.transpose(0, 1).cpu().numpy()
    
    def compute_score(self):
        return self().cpu().numpy()

    def fit(self, user2positives, user2negatives, iterations=200, samples=32, acc_grad=8, 
            lr=0.1, weight_decay=1e-2, margin=1.0):
        # It can be done way more effectively with good dataloader, batches and torch.optim, 
        # but ready solutions are forbidden :(
        
        t = tqdm(range(iterations))
        for i in t:
            u_grad = acc_grad * weight_decay * self.U
            v_grad = acc_grad * weight_decay * self.V
            mean_delta = 0.
            x = self()
            for k in range(acc_grad):
                # Build a batch
                positives = []
                negatives = []
                ranking_weight = []
                pos_mask = []
                neg_mask = []
                for u in range(len(user2positives)):
                    if len(user2positives[u]) != 0:
                        pos = user2positives[u][random.randint(0, len(user2positives[u]) - 1)]
                        pos_mask.append(1)
                        positives.append(pos)
                        if len(user2negatives[u]) != 0:
                            # Finding all high-ranked negatives is very slow, so we sample and estimate rank
                            choosen_negatives = random.sample(user2negatives[u], min(samples, len(user2negatives[u])))
                            bad_negatives = (x[u, choosen_negatives] > x[u, pos]).sum().item()
                            neg_mask.append(1)
                            neg = user2negatives[u][random.randint(0, len(user2negatives[u]) - 1)]
                            negatives.append(neg)
                            if x[u][pos] - x[u][neg] > margin:
                                ranking_weight.append(0) # Margin condition
                            else:
                                ranking_weight.append(np.log(1 + len(user2negatives[u]) * bad_negatives / len(choosen_negatives)))
                            #ranking_weight.append(np.log(1 + len(user2negatives[u]) * len(bad_negatives) / len(choosen_negatives)))
                        
                        else:
                            neg_mask.append(0)
                            negatives.append(0)
                            ranking_weight.append(1)
                    else:
                        pos_mask.append(0)
                        positives.append(0)
                        neg_mask.append(1)
                        ranking_weight.append(1)
                        negatives.append(user2negatives[u][random.randint(0, len(user2negatives[u]) - 1)])

                positives = torch.tensor(positives)
                negatives = torch.tensor(negatives)
                pos_mask = torch.tensor(pos_mask).float()
                neg_mask = torch.tensor(neg_mask).float()
                ranking_weight = torch.tensor(ranking_weight).float()
                
                # Compute gradient
                mean_delta += (x.gather(1, positives.unsqueeze(1)) - x.gather(1, negatives.unsqueeze(1))).mean()
                ranking_weight /= ranking_weight.max() # Fixing stability issues
                v_positives = self.V[:, positives].transpose(0, 1)
                v_negatives = self.V[:, negatives].transpose(0, 1)
                u_grad += ranking_weight[:, None] * (neg_mask[:, None] * v_negatives - pos_mask[:, None] * v_positives)

                v_grad[:, positives] -= ((pos_mask * ranking_weight)[:, None] * self.U).transpose(0, 1)
                v_grad[:, negatives] += ((neg_mask * ranking_weight)[:, None] * self.U).transpose(0, 1)
            self.U -= lr * u_grad / acc_grad
            self.V -= lr * v_grad / acc_grad
            mean_delta /= acc_grad
            t.set_postfix_str(f"Avg delta: {mean_delta:4f} | Avg nabla V: {v_grad.abs().mean():.4f} | L2 Norm: {(self.U**2).mean():.4f}")
        self._build_trees()

warp = WARP(u_users, u_movies, 64)
warp.fit(u2p, u2n)

In [60]:
i_ratings_tensor

tensor([[5., 0., 0.,  ..., 0., 0., 0.],
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.]])

In [61]:
warp()

tensor([[-2.9726, -2.9980, -2.9918,  ..., -4.0777, -4.0973, -4.1130],
        [-3.1142, -3.2395, -3.1724,  ..., -4.1884, -4.2435, -4.2195],
        [-3.1944, -3.1547, -3.1560,  ..., -4.1085, -4.1201, -4.1381],
        ...,
        [-2.6185, -2.5534, -2.5495,  ..., -3.4222, -3.4874, -3.4432],
        [-2.8896, -2.9676, -2.6774,  ..., -3.8913, -3.9428, -3.9211],
        [-2.9340, -3.1158, -2.9771,  ..., -4.0965, -4.1410, -4.1111]])

In [62]:
get_recommendations(4, warp)

['2878    Goldfinger (1964)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '588    Batman (1989)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '2338    Cocoon (1985)',
 '2572    Superman II (1980)',
 '2588    Rocky Horror Picture Show, The (1975)',
 '2882    Fistful of Dollars, A (1964)',
 '1258    Young Frankenstein (1974)',
 '476    Jurassic Park (1993)',
 '2916    Robocop (1987)',
 '3458    Predator (1987)',
 '2332    Pale Rider (1985)',
 '1491    Fifth Element, The (1997)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '1355    Star Trek IV: The Voyage Home (1986)',
 '1192    Star Wars: Episode VI - Return of the Jedi (1983)',
 '3402    Close Encounters of the Third Kind (1977)',
 '928    Adventures of Robin Hood, The (1938)']

In [63]:
get_similars(1, warp)

['0    Toy Story (1995)',
 '1245    Groundhog Day (1993)',
 '3045    Toy Story 2 (1999)',
 '293    Pulp Fiction (1994)',
 "2286    Bug's Life, A (1998)",
 '1656    Good Will Hunting (1997)',
 '711    Wallace & Gromit: The Best of Aardman Animatio...',
 '2918    Who Framed Roger Rabbit? (1988)',
 '2255    Life Is Beautiful (La Vita � bella) (1997)',
 '604    Fargo (1996)',
 '3291    Hoosiers (1986)',
 "523    Schindler's List (1993)",
 '2991    Commitments, The (1991)',
 '1287    When Harry Met Sally... (1989)',
 '589    Silence of the Lambs, The (1991)',
 '1282    Field of Dreams (1989)',
 '3039    Fisher King, The (1991)',
 '2970    Trading Places (1983)',
 '1229    Nikita (La Femme Nikita) (1990)',
 '2199    Few Good Men, A (1992)']